In [69]:
%matplotlib inline
# https://scikit-learn.org/stable/supervised_learning.html
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from datetime import datetime, timedelta, time
import calendar

pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

In [72]:
pff = pd.read_csv('pff_pi.csv', index_col=0)
pff = pff.reset_index().drop(columns='index').reset_index().rename(columns={'index':'game_id'}).set_index('game_id')
pff = pff[['gamenum','Team', 'Opposing Team', 'Season', 'Week', 'Date', 'Time', 'H/A', 'W/L',
       'PA', 'PF', 'OVR', 'OFF', 'PASS', 'PBLK', 'RECV', 'RUN', 'RBLK', 'DEF',
       'RDEF', 'TACK', 'PRSH', 'COV', 'SPEC', 'OVR_opp', 'OFF_opp', 'PASS_opp',
       'PBLK_opp', 'RECV_opp', 'RUN_opp', 'RBLK_opp', 'DEF_opp', 'RDEF_opp',
       'TACK_opp', 'PRSH_opp', 'COV_opp', 'SPEC_opp']]
# pff.iloc[:,9:].drop(columns=['SPEC','SPEC_opp'])
pff.tail(2)

,gamenum,Team,Opposing Team,Season,Week,Date,Time,H/A,W/L,PA,PF,OVR,OFF,PASS,PBLK,RECV,RUN,RBLK,DEF,RDEF,TACK,PRSH,COV,SPEC,OVR_opp,OFF_opp,PASS_opp,PBLK_opp,RECV_opp,RUN_opp,RBLK_opp,DEF_opp,RDEF_opp,TACK_opp,PRSH_opp,COV_opp,SPEC_opp
game_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7166,3583,San Francisco 49Ers,Seattle Seahawks,2019,17,2019-12-29,8:20pm,A,W,21,26,79.9,88.9,88.3,81.6,84.6,72.2,69.2,64.7,65.6,66.5,58.7,65.5,56.3,60.3,70.3,67.1,76.2,67.5,58.9,68.1,39.7,37.0,26.6,61.1,42.6,66.9
7167,3583,Seattle Seahawks,San Francisco 49Ers,2019,17,2019-12-29,8:20pm,H,L,26,21,60.3,70.3,67.1,76.2,67.5,58.9,68.1,39.7,37.0,26.6,61.1,42.6,66.9,79.9,88.9,88.3,81.6,84.6,72.2,69.2,64.7,65.6,66.5,58.7,65.5,56.3


In [90]:
%timeit 
s = pff['Date']
pff['Date'] = pd.to_datetime(s, infer_datetime_format=True)  

In [109]:
# master_df = pff.copy()
df = pff.copy()

In [92]:
teams = master_df['Team'].unique()

Regression loop

In [136]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LassoLars
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

In [138]:
lars_params = {'alpha':[int(x) for x in np.linspace(start = 0.01, stop = 1.0, num = 20)]}

gbr_params = {'loss': ['ls','huber'],
                  'learning_rate': [0.001,0.005, 0.01, 0.05, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6],
                  'max_depth': [int(x) for x in np.linspace(start = 2, stop = 10, num = 5)],
                  'n_estimators': [int(x) for x in np.linspace(start = 2, stop = 10, num = 5)]}

In [139]:
%%time
# initialize lists to store predictions
test_score_predictions = []
holdout_score_predictions = []
test_game_ids = []
holdout_game_ids = []

for i, team in enumerate(teams):
    
    team_test_score_predictions = []
    team_holdout_score_predictions = []
    team_test_game_ids = []
    team_holdout_game_ids = []
    
    # start at 2012 season
    current_date = datetime(2012,9,1)
    stop_date = datetime(2020,1,1)

    # df = master_df[master_df['Team']==team]
    # loop through month
    while current_date < stop_date:
        if current_date.month == 1:
            current_date = datetime(add_month.year, 9, 1)
        else:
            current_date = current_date
        print(str(i), team+':', current_date)

        month_ending = calendar.monthrange(current_date.year,current_date.month)
        month_ending_dt = datetime(current_date.year, current_date.month, month_ending[1])

        # create train/test and holdout dataframes dataframes
        prediction_df = df[(df['Date']<current_date)&(df['Team']==team)].iloc[:,10:].drop(columns=['SPEC','SPEC_opp']).tail(80)
        holdout_df = df[(df['Date']<=month_ending_dt)&(df['Date']>=current_date)&(df['Team']==team)].iloc[:,10:].drop(columns=['SPEC','SPEC_opp'])

        # split y, X arrays
        y = prediction_df.iloc[:,0]
        X = prediction_df.iloc[:,1:]

        # train/test split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False, random_state=42)

        # append game_ids from y_test
        team_test_game_ids.append(y_test.index.values)

        # train model - add cross validate later
        reg = GradientBoostingRegressor().fit(X_train, y_train)

        # Now Cross Validate
        grid_search = GridSearchCV(reg, gbr_params, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=3)
        grid_search.fit(X_train, y_train)
        print("alpha:", grid_search.best_params_)

        gbr_reg = GradientBoostingRegressor(
            loss=grid_search.best_params_['loss'],
            max_depth=grid_search.best_params_['max_depth'],
            n_estimators=grid_search.best_params_['n_estimators'],
            learning_rate=grid_search.best_params_['learning_rate']
        )

        # Train cross validated models
        gbr_reg.fit(X_train, y_train)

        # predict on test set
        test_preds = gbr_reg.predict(X_test) # lars_reg for actual testing
        # append test_preds
        team_test_score_predictions.append(test_preds)

        # grab actual scores from the holdout set
        actual_scores = holdout_df.iloc[:,0]
        # get holdout X dataframe
        X_holdout = holdout_df.iloc[:,1:]

        # predict on holdout set
        holdout_preds = gbr_reg.predict(X_holdout) # lars_reg for actual testing
        # append holdout predictions
        team_holdout_score_predictions.append(holdout_preds)

        # append game_ids from holdout set
        team_holdout_game_ids.append(holdout_df.index.values)

        days_in_month = calendar.monthrange(current_date.year, current_date.month)[1]
        add_month = current_date + timedelta(days = days_in_month)

        current_date = datetime(add_month.year, add_month.month, 1)
        
    team_test_score_predictions_flat = [item for sublist in team_test_score_predictions for item in sublist]
    team_holdout_score_predictions_flat = [item for sublist in team_holdout_score_predictions for item in sublist]
    test_game_ids_flat = [item for sublist in test_game_ids for item in sublist]
    holdout_game_ids_flat = [item for sublist in team_holdout_game_ids for item in sublist]
    
    
    test_score_predictions.append(team_test_score_predictions_flat)
    holdout_score_predictions.append(team_holdout_score_predictions_flat)
    test_game_ids.append(test_game_ids_flat)
    holdout_game_ids.append(holdout_game_ids_flat)

0 Miami Dolphins: 2012-09-01 00:00:00
Fitting 5 folds for each of 550 candidates, totalling 2750 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 1114 tasks      | elapsed:    8.9s
[Parallel(n_jobs=-1)]: Done 2750 out of 2750 | elapsed:   15.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


alpha: {'learning_rate': 0.15, 'loss': 'ls', 'max_depth': 2, 'n_estimators': 8}
0 Miami Dolphins: 2012-10-01 00:00:00
Fitting 5 folds for each of 550 candidates, totalling 2750 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 2750 out of 2750 | elapsed:   12.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


alpha: {'learning_rate': 0.5, 'loss': 'ls', 'max_depth': 2, 'n_estimators': 4}
0 Miami Dolphins: 2012-11-01 00:00:00
Fitting 5 folds for each of 550 candidates, totalling 2750 fits


...

[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 1578 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done 2750 out of 2750 | elapsed:   14.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


alpha: {'learning_rate': 0.3, 'loss': 'huber', 'max_depth': 2, 'n_estimators': 10}
31 Washington Redskins: 2019-12-01 00:00:00
Fitting 5 folds for each of 550 candidates, totalling 2750 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 1936 tasks      | elapsed:    9.9s


alpha: {'learning_rate': 0.3, 'loss': 'huber', 'max_depth': 2, 'n_estimators': 6}
Wall time: 4h 16min 22s


[Parallel(n_jobs=-1)]: Done 2750 out of 2750 | elapsed:   14.4s finished


In [140]:
# flatten list of lists
flat_holdout_score_predictions = [item for sublist in holdout_score_predictions for item in sublist]
flat_holdout_game_ids = [item for sublist in holdout_game_ids for item in sublist]
flat_holdout_game_ids
results = pd.DataFrame(data=flat_holdout_score_predictions, index=flat_holdout_game_ids, columns=['predicted_score'])
results_merge = results.merge(df, left_index=True, right_index=True)
results_merge.head(2)

,predicted_score,gamenum,Team,Opposing Team,Season,Week,Date,Time,H/A,W/L,PA,PF,OVR,OFF,PASS,PBLK,RECV,RUN,RBLK,DEF,RDEF,TACK,PRSH,COV,SPEC,OVR_opp,OFF_opp,PASS_opp,PBLK_opp,RECV_opp,RUN_opp,RBLK_opp,DEF_opp,RDEF_opp,TACK_opp,PRSH_opp,COV_opp,SPEC_opp
3080,14.725165,1540,Miami Dolphins,Houston Texans,2012,1,2012-09-09,1:00pm,A,L,30,10,59.5,53.0,50.7,56.3,57.9,37.7,66.7,67.9,70.1,70.2,68.0,59.7,-,81.1,79.1,90.7,58.3,74.5,64.5,65.2,75.8,78.7,73.3,72.6,67.2,-
3117,22.577273,1558,Miami Dolphins,Oakland Raiders,2012,2,2012-09-16,1:00pm,H,W,13,35,79.9,82.3,76.5,78.4,69.4,84.9,70.1,69.2,90.1,55.6,69.1,55.0,-,60.0,60.4,57.0,48.7,69.7,50.3,51.0,59.5,59.3,65.8,59.6,59.2,-


In [129]:
# results_merge.sort_values(by='gamenum')

In [141]:
from sklearn.metrics import mean_squared_error
math.sqrt(mean_squared_error(results_merge['predicted_score'].values, results_merge['PF'].values))

8.373424374466406

In [142]:
def predicted_results(s):
    if s < 0:
        x = 'L'
    elif s > 0:
        x = 'W'
    return x

def predicted_win_loss(s1, s2):
    if s1 == s2:
        x = 1
    elif s1 != s2:
        x = 0
    return x

# merge results and keep home team rows
analysis = results_merge[results_merge['H/A']=='H'].iloc[:,0:12].merge(results_merge[['gamenum','Opposing Team','predicted_score']], 
                                            left_on=['gamenum','Team'], 
                                            right_on=['gamenum','Opposing Team'], 
                                            suffixes=['_home','_away']).drop_duplicates(subset='gamenum', keep="last")

analysis['point_diff'] = analysis['predicted_score_home'] - analysis['predicted_score_away']
analysis['predicted_results'] = analysis['point_diff'].apply(predicted_results)

print(analysis.apply(lambda x: predicted_win_loss(x['W/L'], x['predicted_results']), axis=1).sum() / len(analysis))
analysis.tail(2)

0.7643849206349206


,predicted_score_home,gamenum,Team,Opposing Team_home,Season,Week,Date,Time,H/A,W/L,PA,PF,Opposing Team_away,predicted_score_away,point_diff,predicted_results
2014,23.286070,3544,Washington Redskins,Philadelphia Eagles,2019,15,2019-12-15,1:00pm,H,L,37,27,Washington Redskins,22.884510,0.401559,W
2015,30.830184,3561,Washington Redskins,New York Giants,2019,16,2019-12-22,1:00pm,H,L,41,35,Washington Redskins,29.823147,1.007037,W


In [143]:
analysis.sort_values(by=['gamenum','Team'])

,predicted_score_home,gamenum,Team,Opposing Team_home,Season,Week,Date,Time,H/A,W/L,PA,PF,Opposing Team_away,predicted_score_away,point_diff,predicted_results
1700,19.542325,1536,New York Giants,Dallas Cowboys,2012,1,2012-09-05,8:30pm,H,L,24,17,New York Giants,32.183475,-12.641150,L
1385,34.343439,1537,Chicago Bears,Indianapolis Colts,2012,1,2012-09-09,1:00pm,H,W,21,41,Chicago Bears,22.385297,11.958142,W
314,10.040000,1538,Cleveland Browns,Philadelphia Eagles,2012,1,2012-09-09,1:00pm,H,L,17,16,Cleveland Browns,24.442822,-14.402822,L
441,20.262236,1539,Detroit Lions,Los Angeles Rams,2012,1,2012-09-09,1:00pm,H,W,23,27,Detroit Lions,26.996438,-6.734202,L
567,24.832690,1540,Houston Texans,Miami Dolphins,2012,1,2012-09-09,1:00pm,H,W,10,30,Houston Texans,14.725165,10.107525,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629,25.744500,3579,Houston Texans,Tennessee Titans,2019,17,2019-12-29,4:25pm,H,L,35,14,Houston Texans,34.115879,-8.371379,L
1637,20.716700,3580,Jacksonville Jaguars,Indianapolis Colts,2019,17,2019-12-29,4:25pm,H,W,20,38,Jacksonville Jaguars,19.177993,1.538707,W
879,31.720771,3581,Los Angeles Rams,Arizona Cardinals,2019,17,2019-12-29,4:25pm,H,W,24,31,Los Angeles Rams,30.629686,1.091086,W
1762,15.039931,3582,New York Giants,Philadelphia Eagles,2019,17,2019-12-29,4:25pm,H,L,34,17,New York Giants,30.602887,-15.562956,L
